In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [2]:
def basic_clean(somestring):
    '''
    Takes in body text of a README and performs a basic clean by first converting it to all lower case letters,
    then normalizes the encoding, and removes any character that isn't a letter, number, or a space.
    Returns the result.
    '''
    basic = somestring.lower()
    basic = unicodedata.normalize('NFKD', basic).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    basic = re.sub(r"[^a-z0-9'\s]", '', basic)
    return basic

def tokenize(somestring):
    '''
    Creates a tokenizer object to tokenize the given string and then returns the tokenized string
    '''
    tokenizer = nltk.tokenize.ToktokTokenizer()
    tokened = tokenizer.tokenize(somestring, return_str=True)    
    return tokened

def lemmatize(somestring):
    '''
    Creates a lemmatize object and then uses it to lemmatize the provided string. Returns the lemmatized string 
    '''
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in somestring.split()]
    article_lemmatized = ' '.join(lemmas)
    return article_lemmatized

def remove_stopwords(string, keep_words=['no', 'not'], exclude_words=[]):
    '''
    This function takes in a string, removes stop_words from the string, and then returns the results.
    The fuction also allows for optional arugments keep_words and exclud_words to modify the stop word list
    * keep_words - a list of words to remove from the standard english stopwords list from nltk.corpus i.e. no or not
    * exclude_words - a list of words to add to the standard english stopwords list from nltk.corpus 
    i.e. ['data', 'science'] to the remove both words when dealing with articles only about data science
    * By default, keep_words includes no and not
    '''
    # Create stopword_list.
    stopword_list = stopwords.words('english')
    # Remove 'keep_words' from stopword_list to keep these in my text.
    stopword_list = set(stopword_list) - set(keep_words)
    # Add in 'exclude_words' to stopword_list.
    stopword_list = stopword_list.union(set(exclude_words))
    # Split words in string.
    words = string.split()
    # Create a list of words from my string with stopwords removed and assign to variable.
    filtered_words = [word for word in words if word not in stopword_list]
    # Join words in the list back into strings and assign to a variable.
    string_without_stopwords = ' '.join(filtered_words)
    return string_without_stopwords


def wrangle(facebook_data_csv_location):
    '''
    This function requires that you download and unzip the file from the link below. File size is over 3gigs
    https://www.propublica.org/datastore/dataset/political-advertisements-from-facebook 
    Once unzipped, use the file path in the fuction to:
    * Load the csv into a pandas dataframe
    * Drops the following columns - 'listbuilding_fundraising_proba', 'targetedness', 'targetings', 'targets', 
    'targeting', 'images', 'lang', 'thumbnail', 'html', 'page_id', 'suppressed', 'entities', 'lower_page'
    * Create a socio_political_fb column that is True if facebook believes that the ad is about soical or political topics, 
    and false if facebook does not
    * Combine the advertiser and paid_for_by columns, with a priority given to paid_for_by values into a advert_by column
    * If advert_by column is null and the page is known, replace the null value with the page value
    * Fills remaining null values with 'unk' for advert_by column
    * Drops any row that has a missing title value (47)
    * Drops the remaining columns with null 
    * Normalizes the message columns by conducting basic_clean, tokenization, removing stop words, and lemmatizing the text
    * Splits the data into train, validate, and test sets that are stratified on socio_political_fb
    * returns train, validate, test
    '''

    df = pd.read_csv(facebook_data_csv_location)
    df.drop(columns=['listbuilding_fundraising_proba', 'targetedness', 'targetings', 'targets', 
    'targeting', 'images', 'lang', 'thumbnail', 'html', 'page_id', 'suppressed', 'entities', 'lower_page'], inplace=True)
    df['socio_political_fb'] = df['paid_for_by'].notnull()
    df['advert_by'] = np.where(df.paid_for_by.isnull(), df.advertiser, df.paid_for_by)
    df.advert_by = np.where(df.advert_by.isnull(), df.page, df.advert_by)
    df['advert_by'].fillna('unk', inplace=True)
    df.dropna(subset=['title'], inplace= True)
    df.dropna(axis=1, inplace=True)
    df['message'] = df['message'].str.replace(r'(<.+\")', "")
    df['message'] = df['message'].str.replace("</span>", "")
    df['message'] = df['message'].str.replace("</p>", "")
    df['message'] = df['message'].str.replace("<p>", "")
    df['message'] = df['message'].apply(basic_clean).apply(tokenize).apply(remove_stopwords).apply(lemmatize)
    train_validate, test = train_test_split(df, 
                                        stratify=df.socio_political_fb, 
                                        test_size=.2, 
                                        random_state=333)

    train, validate = train_test_split(train_validate, 
                                       stratify=train_validate.socio_political_fb, 
                                       test_size=.25,
                                       random_state=333)
    return train, validate, test

In [3]:
df = pd.read_csv('facebookads.csv')
df.drop(columns=['listbuilding_fundraising_proba', 'targetedness', 'targetings', 'targets', 
'targeting', 'images', 'lang', 'thumbnail', 'html', 'page_id', 'suppressed', 'entities', 'lower_page'], inplace=True)
df['socio_political_fb'] = df['paid_for_by'].notnull()
df['advert_by'] = np.where(df.paid_for_by.isnull(), df.advertiser, df.paid_for_by)
df.advert_by = np.where(df.advert_by.isnull(), df.page, df.advert_by)
df['advert_by'].fillna('unk', inplace=True)
df.dropna(subset=['title'], inplace= True)
df.dropna(axis=1, inplace=True)
df['message'] = df['message'].str.replace(r'(<.+\")', "")
df['message'] = df['message'].str.replace("</span>", "")
df['message'] = df['message'].str.replace("</p>", "")
df['message'] = df['message'].str.replace("<p>", "")
df['message'] = df['message'].apply(basic_clean).apply(tokenize).apply(remove_stopwords).apply(lemmatize)
df['title_cleaned'] = df['title'].apply(basic_clean)
df['advert_by_cleaned'] = df['title'].apply(basic_clean)


In [4]:
df.dtypes

id                        object
political                  int64
not_political              int64
title                     object
message                   object
created_at                object
updated_at                object
impressions                int64
political_probability    float64
socio_political_fb          bool
advert_by                 object
title_cleaned             object
advert_by_cleaned         object
dtype: object

In [5]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['updated_at'] = pd.to_datetime(df['updated_at'])
df['known_ad_length'] = df['updated_at'] - df['created_at'] 

df['known_ad_length'] = df['known_ad_length'] // pd.Timedelta('1d')

df['message_length'] = df['message'].str.split().str.len()
df

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,socio_political_fb,advert_by,title_cleaned,advert_by_cleaned,known_ad_length,message_length
0,23844966472130541,1,0,Stop Republicans,election changing rapidly many democrat terrif...,2020-04-28 18:29:51.893295+00:00,2020-04-28 20:00:18.291895+00:00,1,0.999563,True,Stop Republicans,stop republicans,stop republicans,0,51
1,23844607318170064,0,1,Planned Parenthood,uncertain time shouldnt impact access dependab...,2020-05-03 21:21:17.022063+00:00,2020-05-03 23:00:14.598025+00:00,1,0.426290,True,Planned Parenthood Federation of America,planned parenthood,planned parenthood,0,11
2,23844646560350048,0,0,No Kid Hungry,givingtuesdaya happening may 5th response covi...,2020-05-03 21:50:12.097628+00:00,2020-05-03 23:00:14.717141+00:00,1,0.625368,True,No Kid Hungry,no kid hungry,no kid hungry,0,35
3,23844775979230727,1,0,Kirsten Gillibrand,want take back senate retire mitch mcconnell s...,2020-04-27 23:44:32.620687+00:00,2020-04-28 19:48:27.917779+00:00,3,1.000000,True,Gillibrand for Senate,kirsten gillibrand,kirsten gillibrand,0,43
4,23844529028560107,0,2,Learn How to Eliminate Discomfort Today -->,neck pain caused several factor effectively tr...,2020-03-28 15:38:36.167560+00:00,2020-04-28 19:52:56.791781+00:00,3,0.701307,False,unk,learn how to eliminate discomfort today,learn how to eliminate discomfort today,31,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222181,23844404769680021,2,0,Shahid Buttar for Congress,housing homeless life working people lowincome...,2020-04-02 15:47:15.164735+00:00,2020-04-29 10:21:06.065500+00:00,4,0.999264,True,Shahid Buttar for Congress Committee,shahid buttar for congress,shahid buttar for congress,26,19
222182,6175667699907,0,0,Catholic Relief Services,cr ground preventing responding covid19 outbre...,2020-05-10 20:24:20.711342+00:00,2020-05-10 22:00:11.252752+00:00,1,0.687591,True,CATHOLIC RELIEF SERVICES - UNITED STATES CONFE...,catholic relief services,catholic relief services,0,19
222183,23844574679700670,2,0,Elizabeth Warren,running president one elizabeth favorite thing...,2020-04-29 23:30:26.644050+00:00,2020-04-30 14:00:06.087094+00:00,1,0.999989,True,"WARREN DEMOCRATS, INC.",elizabeth warren,elizabeth warren,0,42
222184,23844247053180740,0,1,Be a catalyst for change.,antioch mba feature mutual importance people p...,2020-03-23 23:00:46.971931+00:00,2020-04-29 23:30:55.279836+00:00,2,0.825052,False,unk,be a catalyst for change,be a catalyst for change,37,9


In [6]:
df['title_cleaned'] = df['title'].apply(basic_clean).apply(tokenize).apply(remove_stopwords).apply(lemmatize)
df['advert_by_cleaned'] = df['advert_by'].apply(basic_clean).apply(tokenize).apply(remove_stopwords).apply(lemmatize)

In [7]:
df.head(5)

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,socio_political_fb,advert_by,title_cleaned,advert_by_cleaned,known_ad_length,message_length
0,23844966472130541,1,0,Stop Republicans,election changing rapidly many democrat terrif...,2020-04-28 18:29:51.893295+00:00,2020-04-28 20:00:18.291895+00:00,1,0.999563,True,Stop Republicans,stop republican,stop republican,0,51
1,23844607318170064,0,1,Planned Parenthood,uncertain time shouldnt impact access dependab...,2020-05-03 21:21:17.022063+00:00,2020-05-03 23:00:14.598025+00:00,1,0.426290,True,Planned Parenthood Federation of America,planned parenthood,planned parenthood federation america,0,11
2,23844646560350048,0,0,No Kid Hungry,givingtuesdaya happening may 5th response covi...,2020-05-03 21:50:12.097628+00:00,2020-05-03 23:00:14.717141+00:00,1,0.625368,True,No Kid Hungry,no kid hungry,no kid hungry,0,35
3,23844775979230727,1,0,Kirsten Gillibrand,want take back senate retire mitch mcconnell s...,2020-04-27 23:44:32.620687+00:00,2020-04-28 19:48:27.917779+00:00,3,1.000000,True,Gillibrand for Senate,kirsten gillibrand,gillibrand senate,0,43
4,23844529028560107,0,2,Learn How to Eliminate Discomfort Today -->,neck pain caused several factor effectively tr...,2020-03-28 15:38:36.167560+00:00,2020-04-28 19:52:56.791781+00:00,3,0.701307,False,unk,learn eliminate discomfort today,unk,31,10


In [8]:
df['full_ad'] = df['message'] + " " + df['title_cleaned']
df['full_ad']

0         election changing rapidly many democrat terrif...
1         uncertain time shouldnt impact access dependab...
2         givingtuesdaya happening may 5th response covi...
3         want take back senate retire mitch mcconnell s...
4         neck pain caused several factor effectively tr...
                                ...                        
222181    housing homeless life working people lowincome...
222182    cr ground preventing responding covid19 outbre...
222183    running president one elizabeth favorite thing...
222184    antioch mba feature mutual importance people p...
222185               little desperate together thank reader
Name: full_ad, Length: 222139, dtype: object

In [9]:
df['full_ad_length'] = df['full_ad'].str.split().str.len()
df[['full_ad_length', 'message_length']]

,full_ad_length,message_length
0,53,51
1,13,11
2,38,35
3,45,43
4,14,10
...,...,...
222181,22,19
222182,22,19
222183,44,42
222184,11,9


In [10]:
df['socio_political_fb'] = df['socio_political_fb'].replace({True:'Political', False:'Not Political'})

In [12]:
df['known_ad_time'] = df['updated_at'] - df['created_at'] 
df['known_ad_time'] = df['known_ad_time'] // pd.Timedelta('1d') + 1
df['known_ad_time']

0          1
1          1
2          1
3          1
4         32
          ..
222181    27
222182     1
222183     1
222184    38
222185     3
Name: known_ad_time, Length: 222139, dtype: int64

In [1]:
from wrangle import wrangle
train, validate, test = wrangle('facebookads.csv')
train

,id,political,not_political,title,message,created_at,updated_at,impressions,political_probability,socio_political_fb,advert_by,known_ad_time,message_length,full_ad,full_ad_length
132967,23842810769120558,0,0,end citizen united,koch said spend 400 million preserve republica...,2018-03-01 19:50:38.892007+00:00,2018-03-01 19:50:38.910542+00:00,2,0.973059,Not Political,Arizona Democratic Party,1,9,koch said spend 400 million preserve republica...,12
171238,hyperfeed_story_id_5cdb616c650cc2c08453510,1,0,jay inslee,simple get jay debate stage june need 65000 do...,2019-05-15 00:47:15.677809+00:00,2019-05-15 02:08:15.943151+00:00,1,0.999831,Political,INSLEE FOR AMERICA,1,10,simple get jay debate stage june need 65000 do...,12
157472,6113998509676,0,0,athens everyone,vote november 6 check voter guide information ...,2018-10-26 01:52:16.250883+00:00,2018-10-26 01:52:16.250883+00:00,1,0.984821,Political,Athens for Everyone,1,9,vote november 6 check voter guide information ...,11
18371,23843613746350544,2,2,ph pennsylvania horticultural society,httpsbitly34h5kvxa,2019-09-30 08:21:12.454423+00:00,2019-10-01 17:05:33.831098+00:00,1,0.717423,Not Political,PHS : Pennsylvania Horticultural Society,2,1,httpsbitly34h5kvxa ph pennsylvania horticultur...,5
221209,23844574775710064,0,0,planned parenthood,planned parenthood expert make life easier eve...,2020-04-28 00:09:46.224171+00:00,2020-04-28 15:00:14.714633+00:00,1,0.617928,Political,Planned Parenthood Federation of America,1,19,planned parenthood expert make life easier eve...,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204046,23844309334770475,0,0,yes b better choice,casa familiara support yes b provides affordab...,2020-02-11 18:36:10.972937+00:00,2020-04-21 16:19:28.717269+00:00,1,0.929575,Political,Yes on B for a Better Choice: A coalition of f...,70,9,casa familiara support yes b provides affordab...,13
15498,23843659201280353,1,0,john w hickenlooper,' running senate make sure leader washington '...,2019-10-12 21:17:49.405876+00:00,2019-10-13 00:23:59.421235+00:00,1,0.999979,Political,Hickenlooper for Colorado,1,17,' running senate make sure leader washington '...,20
75422,6081931914599,4,0,moveonorg,jared kushner brought real estate business whi...,2018-03-06 06:20:26.287671+00:00,2018-03-09 05:07:01.686611+00:00,2,0.998760,Not Political,MoveOn.org,3,20,jared kushner brought real estate business whi...,21
82449,23842805912770028,4,0,kyrsten sinema,replace jeff flake someone protect health care...,2018-05-30 03:19:11.422448+00:00,2018-06-04 00:25:05.966551+00:00,1,0.999997,Not Political,Kyrsten Sinema,5,28,replace jeff flake someone protect health care...,30
